In [2]:
import requests
from bs4 import BeautifulSoup

# 设置请求头，模拟浏览器访问
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'
}

# 目标URL（需根据需要替换）
url = 'https://flights.ctrip.com/online/list/round-akl-chc?depdate=2024-11-29_2024-12-02&cabin=y_s&adult=1&child=0&infant=0'

# 发送请求
response = requests.get(url, headers=headers)

# 检查请求是否成功
if response.status_code == 200:
    print("请求成功！")
else:
    print("请求失败，状态码：", response.status_code)

# 解析HTML
soup = BeautifulSoup(response.text, 'html.parser')

# 提取机票信息（根据实际HTML结构进行调整）
flights = soup.find_all('div', class_='flight-info')

for flight in flights:
    airline = flight.find('span', class_='airline-name').text
    price = flight.find('span', class_='price').text
    departure_time = flight.find('span', class_='departure-time').text
    arrival_time = flight.find('span', class_='arrival-time').text
    
    print(f"航空公司：{airline}, 价格：{price}, 起飞时间：{departure_time}, 到达时间：{arrival_time}")

请求成功！


In [3]:
# ... existing imports ...
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 设置Chrome选项
chrome_options = Options()
chrome_options.add_argument('--headless')  # 无头模式
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument(f'user-agent={headers["User-Agent"]}')

# 初始化浏览器
driver = webdriver.Chrome(options=chrome_options)

try:
    # 访问网页
    driver.get(url)
    
    # 等待页面加载（等待航班信息出现）
    wait = WebDriverWait(driver, 20)
    flight_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[class*="flight-item"]')))
    
    # 给页面一点额外时间加载
    time.sleep(5)
    
    # 获取页面内容
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # 提取航班信息（需要根据实际HTML结构调整选择器）
    flights = soup.select('[class*="flight-item"]')
    
    for flight in flights:
        try:
            airline = flight.select_one('[class*="airline-name"]').text.strip()
            price = flight.select_one('[class*="price"]').text.strip()
            departure_time = flight.select_one('[class*="departure-time"]').text.strip()
            arrival_time = flight.select_one('[class*="arrival-time"]').text.strip()
            
            print(f"航空公司：{airline}, 价格：{price}, 起飞时间：{departure_time}, 到达时间：{arrival_time}")
        except AttributeError as e:
            print(f"提取数据时出错: {e}")

finally:
    driver.quit()

提取数据时出错: 'NoneType' object has no attribute 'text'
提取数据时出错: 'NoneType' object has no attribute 'text'
提取数据时出错: 'NoneType' object has no attribute 'text'
提取数据时出错: 'NoneType' object has no attribute 'text'
提取数据时出错: 'NoneType' object has no attribute 'text'
提取数据时出错: 'NoneType' object has no attribute 'text'


In [4]:
try:
    # 访问网页
    driver.get(url)
    
    # 等待页面加载（增加等待时间）
    wait = WebDriverWait(driver, 30)
    flight_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[class*="flight-list"]')))
    
    # 增加等待时间
    time.sleep(10)
    
    # 获取页面内容
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # 打印页面内容，用于调试
    print("页面内容片段：")
    print(soup.prettify()[:1000])
    
    # 使用更具体的选择器
    flights = soup.select('div[class*="flight-list"] div[class*="flight-item"]')
    print(f"找到 {len(flights)} 个航班项目")
    
    for flight in flights:
        try:
            # 打印当前航班项目的HTML，用于调试
            print("\n当前航班HTML:")
            print(flight.prettify())
            
            # 使用更多可能的选择器
            airline = (
                flight.select_one('span[class*="airline"]') or 
                flight.select_one('div[class*="airline"]') or 
                flight.select_one('[class*="carrier"]')
            )
            price = (
                flight.select_one('span[class*="price"]') or 
                flight.select_one('div[class*="price"]') or 
                flight.select_one('[class*="amount"]')
            )
            departure_time = (
                flight.select_one('[class*="depart"]') or 
                flight.select_one('[class*="departure"]')
            )
            arrival_time = (
                flight.select_one('[class*="arrival"]') or 
                flight.select_one('[class*="arrive"]')
            )
            
            # 检查是否找到所有元素
            if all([airline, price, departure_time, arrival_time]):
                print(f"航空公司：{airline.text.strip()}, "
                      f"价格：{price.text.strip()}, "
                      f"起飞时间：{departure_time.text.strip()}, "
                      f"到达时间：{arrival_time.text.strip()}")
            else:
                print("未能找到完整的航班信息")
                print(f"airline: {airline}")
                print(f"price: {price}")
                print(f"departure_time: {departure_time}")
                print(f"arrival_time: {arrival_time}")
                
        except Exception as e:
            print(f"处理航班信息时出错: {str(e)}")

finally:
    driver.quit()

MaxRetryError: HTTPConnectionPool(host='localhost', port=63852): Max retries exceeded with url: /session/a0ed0e10b6ceaace8b5b64f44ebf5162/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000014D4A871670>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 设置Chrome选项
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument(f'user-agent={headers["User-Agent"]}')

# 使用 webdriver_manager 自动管理 ChromeDriver
service = Service(ChromeDriverManager().install())

# 初始化浏览器
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    # 其余代码保持不变
    driver.get(url)
    # ... 后续代码 ...

finally:
    driver.quit()